In [1]:
from tensorflow.keras.layers import Dense,InputLayer, Conv2D, MaxPooling2D, Dense
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import layers, models
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("runs/experiment1")

In [2]:
# Cargar el dataset ya dividido en train/val/test usando el parámetro split de TFDS
(train, val, test), data_info = tfds.load("malaria",with_info=True,
                                        split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
                                        shuffle_files=True)

In [3]:
data_info

tfds.core.DatasetInfo(
    name='malaria',
    full_name='malaria/1.0.0',
    description="""
    The Malaria dataset contains a total of 27,558 cell images with equal instances
    of parasitized and uninfected cells from the thin blood smear slide images of
    segmented cells.
    """,
    homepage='https://lhncbc.nlm.nih.gov/publication/pub9932',
    data_dir='C:\\Users\\vpn\\tensorflow_datasets\\malaria\\1.0.0',
    file_format=tfrecord,
    download_size=337.08 MiB,
    dataset_size=317.62 MiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    nondeterministic_order=False,
    splits={
        'train': <SplitInfo num_examples=27558, num_shards=4>,
    },
    citation="""@article{rajaraman2018pre,
      title={Pre-trained convolutional neural networks as feature extractors toward
      improved mal

In [4]:

def resize_only(example):
    image = example['image']
    label = example['label']
    # Redimensionar
    image = tf.image.resize(image, (224, 224))
    # Normalizar (opcional pero recomendado)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

AUTOTUNE = tf.data.AUTOTUNE

train_dataset = (train
                .map(resize_only, num_parallel_calls=AUTOTUNE)
                .batch(32)
                .prefetch(AUTOTUNE))

val_dataset = (val
                .map(resize_only, num_parallel_calls=AUTOTUNE)
                .batch(32)
                .prefetch(AUTOTUNE))

test_dataset = (test
                .map(resize_only, num_parallel_calls=AUTOTUNE)
                .batch(32)
                .prefetch(AUTOTUNE))


In [5]:
#Definimos el modelo CNN
model = models.Sequential([
    # Entrada
    layers.Input(shape=(224, 224, 3)),
    #Bloque 1
    layers.Conv2D(10, (3,3), strides=1, padding='same',
                activation='relu', use_bias=True, name='conv1'),
    layers.Conv2D(10, (3,3), strides=1, padding='same',
                activation='relu', use_bias=True, name='conv2'),
    layers.MaxPooling2D((2,2), name='pool1'),#Reduce resolucion a la mitad
    # Bloque 2
    layers.Conv2D(10, (3,3), strides=1, padding='same',
                activation='relu', use_bias=True, name='conv3'),
    layers.Conv2D(10, (3,3), strides=1, padding='same',
                activation='relu', use_bias=True, name='conv4'),
    layers.MaxPooling2D((2,2), name='pool2'),#Reduce resolucion a la mitad
    #Reducción y capa densa
    layers.GlobalAveragePooling2D(name='gap'), #Promedia cada mapa en un vector de tamaño (batch_size, n_classes)
    layers.Dense(128, activation='relu', use_bias=True, name='fc1'),#Capa densa de 128 neyronas que procesa el vector de salida del gap
    layers.Dropout(0.3, name='dropout'),#Se apagan algunas neuronas el 30% en entrenamiento se encienden
    #Capa de salida (softmax porque es una clasificación)
    layers.Dense(2, activation='softmax', use_bias=True, name='output')
])

#Compilacion del modelo
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='sparse_categorical_crossentropy',   # porque las etiquetas son enteros 0/1
            metrics=['accuracy'])

In [6]:
#Resumen del modelo
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 224, 224, 10)      280       
                                                                 
 conv2 (Conv2D)              (None, 224, 224, 10)      910       
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 10)      0         
                                                                 
 conv3 (Conv2D)              (None, 112, 112, 10)      910       
                                                                 
 conv4 (Conv2D)              (None, 112, 112, 10)      910       
                                                                 
 pool2 (MaxPooling2D)        (None, 56, 56, 10)        0         
                                                                 
 gap (GlobalAveragePooling2  (None, 10)                0

In [7]:
#Entrenamiento
model.fit(
  train_dataset,
  validation_data = val_dataset,
  epochs = 20,
  callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True),
        tf.keras.callbacks.ModelCheckpoint('best_malaria_cnn.h5', save_best_only=True)
    ]
)

Epoch 1/20



689/689 [==============================] - 114s 163ms/step - loss: 0.6545 - accuracy: 0.6009 - val_loss: 0.5446 - val_accuracy: 0.7268
Epoch 2/20


c:\Users\vpn\Documents\GitHub\Ciencia-de-datos\.venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


232/689 [=========>....................] - ETA: 1:12 - loss: 0.3984 - accuracy: 0.8512

KeyboardInterrupt: 

In [ ]:
#Acuracy
save = load_model("best_malaria_cnn.h5")
test_loss, test_acc = save.evaluate(test_dataset)
print(f"Exactitud en test: {test_acc:.3f}")

87/87 [==============================] - 5s 52ms/step - loss: 0.1371 - accuracy: 0.9586
Exactitud en test: 0.959


In [ ]:
# Tomar un batch del dataset
example_batch = next(iter(test_dataset.take(1)))

# Extraer una sola imagen (posición 0)
img = example_batch[0][0]  # primera imagen del batch
label = example_batch[1][0]  # su etiqueta

# Redimensionar y normalizar
img = tf.image.resize(img, (224, 224)) / 255.0
img = tf.expand_dims(img, axis=0)  # añadir dimensión batch

# Predicción
pred = save.predict(img)
print("Predicción:", pred)
print("Etiqueta real:", label.numpy())

1/1 [==============================] - 0s 82ms/step
Predicción: [[0.12280191 0.87719804]]
Etiqueta real: 1
